# Data Preparation

### Read Data

In [1]:
import numpy as np
import pandas as pd
import os

dataset_path = os.path.join('data', '2022_season2.csv')
rental_data_path = os.path.join('data', 'Daejeon_tashu_rental_office_list_.xlsx')

dataset = pd.read_csv(dataset_path,
                      parse_dates=['대여일시', '반납일시'],
                      infer_datetime_format=True)

rental_data = pd.read_excel(rental_data_path, 
                             sheet_name='대여소정보(시즌2)', 
                             header=0,
                             index_col='번호',
                             dtype={
                                 '구분': str,
                                 '구': str,
                                 '동': str,
                                 '위도': float,
                                 '경도': float,
                                 '사용여부': str
                             })

### Add Rental time column

In [2]:
dataset['rental_time'] = dataset['반납일시']-dataset['대여일시']
dataset['rental_time'] = dataset['rental_time'].apply(lambda x: x.seconds)
dataset.tail()

,대여스테이션,대여일시,반납스테이션,반납일시,이동거리,rental_time
1330249,1088,2022-12-31 23:58:46,64,2023-01-01 00:18:27,1100,1181
1330250,763,2022-12-31 23:58:57,198,2023-01-01 00:05:49,500,412
1330251,734,2022-12-31 23:58:59,398,2023-01-01 00:54:14,4900,3315
1330252,426,2022-12-31 23:59:41,102,2023-01-01 00:19:26,2500,1185
1330253,876,2022-12-31 23:59:45,125,2023-01-01 00:08:20,1000,515


In [3]:
# if Rental_time < 500 secs and rental/return stations are same, the data will be removed.
dataset = dataset.loc[(dataset['rental_time'] > 300) & (dataset['대여스테이션'] != dataset['반납스테이션'])]
dataset

,대여스테이션,대여일시,반납스테이션,반납일시,이동거리,rental_time
47,1,2022-07-23 10:12:20,985,2022-07-23 10:20:39,500,499
48,985,2022-07-23 10:23:19,885,2022-07-23 10:29:18,100,359
49,985,2022-07-23 10:23:24,885,2022-07-23 10:29:30,100,366
50,885,2022-07-23 10:33:08,1,2022-07-23 10:39:16,500,368
51,885,2022-07-23 10:33:17,1,2022-07-23 10:39:25,500,368
...,...,...,...,...,...,...
1330249,1088,2022-12-31 23:58:46,64,2023-01-01 00:18:27,1100,1181
1330250,763,2022-12-31 23:58:57,198,2023-01-01 00:05:49,500,412
1330251,734,2022-12-31 23:58:59,398,2023-01-01 00:54:14,4900,3315
1330252,426,2022-12-31 23:59:41,102,2023-01-01 00:19:26,2500,1185


### Drop the not used station

In [4]:
not_used = rental_data.loc[rental_data['사용여부'] == 'N']
rental_data = rental_data.loc[rental_data['사용여부'] == 'Y']
rental_data = rental_data.drop(columns=['사용여부'])
rental_data

,구분,대여소명,구,동,위도,경도
번호,,,,,,
1,타슈2,타슈관제센터,유성구,문지동,36.389501,127.409377
3,타슈2,탄방동 한사랑병원,서구,탄방동,36.348446,127.390052
4,타슈2,한가람아파트 2동,서구,탄방동,36.348201,127.390409
5,타슈2,목련아파트 103동,서구,탄방동,36.350037,127.390402
6,타슈2,크로바아파트 103동,서구,둔산동,36.352480,127.390385
...,...,...,...,...,...,...
1176,타슈2,가양동 올댓 앞,동구,가양동,36.346604,127.444700
1177,타슈2,가양동 트윈스아파트,동구,가양동,36.345427,127.451728
1178,타슈2,가팔어린이공원,동구,가양동,36.344248,127.449159


In [5]:
not_used.index

Int64Index([   2,    9,   28,  121,  180,  245,  252,  345,  350,  373,  402,
             429,  436,  441,  450,  458,  460,  475,  510,  530,  544,  567,
             612,  614,  635,  637,  654,  688,  722,  725,  742,  762,  792,
             853,  854,  855,  858,  915,  919,  921,  924,  932,  955,  977,
            1022, 1035, 1048, 1049, 1089, 1096, 1097, 1098, 1099, 1105, 1121,
            1134, 1144, 1152, 1164, 1174],
           dtype='int64', name='번호')

In [6]:
def remove_from_index_array(target: pd.Series, index: np.ndarray) -> pd.Series:
    c = target.copy()
    for i, v in target.items():
        if v in index:      # if index array contains v
            c[i] = False    # remove this row.
        else:               # if not contains v
            c[i] = True     # keep this row.
    return c

In [7]:
dataset = dataset.loc[remove_from_index_array(dataset['대여스테이션'], not_used.index)]
dataset = dataset.loc[remove_from_index_array(dataset['반납스테이션'], not_used.index)]
dataset

,대여스테이션,대여일시,반납스테이션,반납일시,이동거리,rental_time
47,1,2022-07-23 10:12:20,985,2022-07-23 10:20:39,500,499
48,985,2022-07-23 10:23:19,885,2022-07-23 10:29:18,100,359
49,985,2022-07-23 10:23:24,885,2022-07-23 10:29:30,100,366
50,885,2022-07-23 10:33:08,1,2022-07-23 10:39:16,500,368
51,885,2022-07-23 10:33:17,1,2022-07-23 10:39:25,500,368
...,...,...,...,...,...,...
1330249,1088,2022-12-31 23:58:46,64,2023-01-01 00:18:27,1100,1181
1330250,763,2022-12-31 23:58:57,198,2023-01-01 00:05:49,500,412
1330251,734,2022-12-31 23:58:59,398,2023-01-01 00:54:14,4900,3315
1330252,426,2022-12-31 23:59:41,102,2023-01-01 00:19:26,2500,1185


### Save processed data

In [8]:
dataset.to_csv(os.path.join('preprocessed', '2022_season2_processed.csv'))
rental_data.to_csv(os.path.join('preprocessed', 'season2_rental_office.csv'))